In [1]:
import pandas as pd
import json
import numpy as np

In [2]:
# Read the CSV file into a DataFrame
csv101_df = pd.read_csv('101.csv')

print(len(csv101_df))

105


In [3]:
# Read the JSONL file
allArtworks = []
with open(r"api-data\getting-started\allArtworks.jsonl", "r") as rows:
    for row in rows:
        allArtworks.append(json.loads(row))

In [4]:
# Create a DataFrame from the list of JSON objects
allArtworks_df = pd.DataFrame(allArtworks)

In [5]:
# Convert the artist_title column so it's case-insensitive
allArtworks_df['artist_title_lower'] = allArtworks_df['artist_title'].str.lower()

# Change column name from 'department_title' to 'classification'
allArtworks_df = allArtworks_df.rename(columns={"department_title": "classification"})

# Classifications to be removed
class_removal = ['Photography and Media', 'Textiles','Applied Arts of Europe','AIC Archives']

# Remove unwanted classifications
allArtworks_df = allArtworks_df[~allArtworks_df['classification'].isin(class_removal)]

allArtworks_df.head()

,id,title,main_reference_number,classification,artist_title,artist_title_lower
0,4,Priest and Boy,1880.1,Prints and Drawings,Lawrence Carmichael Earle,lawrence carmichael earle
1,9,"Interior of St. Mark's, Venice",1887.232,Arts of the Americas,David Dalhoff Neal,david dalhoff neal
2,11,Self-Portrait,1887.234,Arts of the Americas,Walter Shirlaw,walter shirlaw
3,16,The Fall of the Giants,1887.249,Prints and Drawings,Salvator Rosa,salvator rosa
4,19,"View of Ponte Lugano on the Anio, from Views o...",1887.252,Prints and Drawings,Giovanni Battista Piranesi,giovanni battista piranesi


In [6]:
# Add 'artist_title_lower' to make case-insensitive 
csv101_df["artist_title_lower"] = csv101_df['nameP'].str.lower()
csv101_df["artist_title_lower"] = csv101_df["artist_title_lower"].str.strip()

csv101_df.head()

,nameP,colloq_name,last_name,lifespanP,separator,bio,artist_title_lower
0,Pablo Picasso,Picasso,Picasso,1881-1973,1,"Love him or hate him, Pablo Picasso changed it...",pablo picasso
1,Giotto Di Bondone,Giotto,Di Bondone,c.1267-1337,1,The greatest renovator of early European paint...,giotto di bondone
2,Leonardo Da Vinci,Leonardo,Da Vinci,1452-1519,1,There is no artist more legendary than Leonard...,leonardo da vinci
3,Paul Cézanne,Cézanne,Cézanne,1839-1906,1,“Cézanne is the father of us all”. This lapida...,paul cézanne
4,Rembrandt Van Rijn,Rembrandt,Rijn,1606-1669,2,"The fascinating, magnetic interplay of light a...",rembrandt van rijn


In [7]:
merged_df = allArtworks_df.merge(csv101_df, on='artist_title_lower', how='inner')
merged_df['museum_id'] = 'AIC'

merged_df.head(3)


,id,title,main_reference_number,classification,artist_title,artist_title_lower,nameP,colloq_name,last_name,lifespanP,separator,bio,museum_id
0,74,Three Oriental Figures (Jacob and Laban?),1887.462,Prints and Drawings,Rembrandt van Rijn,rembrandt van rijn,Rembrandt Van Rijn,Rembrandt,Rijn,1606-1669,2,"The fascinating, magnetic interplay of light a...",AIC
1,4196,"The Artist's Mother Seated at a Table, Looking...",1956.981,Prints and Drawings,Rembrandt van Rijn,rembrandt van rijn,Rembrandt Van Rijn,Rembrandt,Rijn,1606-1669,2,"The fascinating, magnetic interplay of light a...",AIC
2,9596,The First Oriental Head,1923.1108,Prints and Drawings,Rembrandt van Rijn,rembrandt van rijn,Rembrandt Van Rijn,Rembrandt,Rijn,1606-1669,2,"The fascinating, magnetic interplay of light a...",AIC


In [12]:
artworkAIC = merged_df[['colloq_name', 'title', 'museum_id', 'classification']]
artworkAIC['creation_date'] = np.nan

artworkAIC.head()

C:\Users\rplea\AppData\Local\Temp\ipykernel_17768\628274986.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  artworkAIC['creation_date'] = np.nan


,colloq_name,title,museum_id,classification,creation_date
0,Rembrandt,Three Oriental Figures (Jacob and Laban?),AIC,Prints and Drawings,NaN
1,Rembrandt,"The Artist's Mother Seated at a Table, Looking...",AIC,Prints and Drawings,NaN
2,Rembrandt,The First Oriental Head,AIC,Prints and Drawings,NaN
3,Rembrandt,Kostverloren Castle in Decay,AIC,Prints and Drawings,NaN
4,Rembrandt,Self-Portrait in a Flat Cap and Embroidered Dress,AIC,Prints and Drawings,NaN


In [13]:
# Convert DataFrame to CSV file
artworkAIC.to_csv('aic.csv', index=False)

In [9]:
# Count the number of classifications
unique_counts = artworkAIC['classification'].value_counts()
print(unique_counts)

classification
Prints and Drawings                                  3595
Modern Art                                            120
Painting and Sculpture of Europe                      115
Contemporary Art                                       56
Ryerson and Burnham Libraries Special Collections      18
Arts of the Americas                                   12
Name: count, dtype: int64
